# Dedoc

本示例演示了如何将 `Dedoc` 与 `LangChain` 结合作为 `DocumentLoader` 使用。

## 概述

[Dedoc](https://dedoc.readthedocs.io) 是一个[开源](https://github.com/ispras/dedoc)
库/服务，可从各种格式的文件中提取文本、表格、附件和文档结构（例如，标题、列表项等）。

`Dedoc` 支持 `DOCX`、`XLSX`、`PPTX`、`EML`、`HTML`、`PDF`、图片等。
支持的文件格式的完整列表可以在[此处](https://dedoc.readthedocs.io/en/latest/#id1)找到。


### 集成详情

| 类                                                                                                                                                   | 包                                                                                           | 本地 | 可序列化 | JS 支持 |
|:-----------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------|:-----:|:------------:|:----------:|
| [DedocFileLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.dedoc.DedocFileLoader.html)       | [langchain_community](https://python.langchain.com/api_reference/community/index.html) |   ❌   |     beta     |     ❌      |
| [DedocPDFLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.DedocPDFLoader.html)           | [langchain_community](https://python.langchain.com/api_reference/community/index.html) |   ❌   |     beta     |     ❌      |
| [DedocAPIFileLoader](https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.dedoc.DedocAPIFileLoader.html) | [langchain_community](https://python.langchain.com/api_reference/community/index.html) |   ❌   |     beta     |     ❌      |


### 加载器功能

提供了延迟加载和异步加载的方法，但实际上，文档加载是同步执行的。

|       来源       | 文档延迟加载 | 异步支持 |
|:------------------:|:---------------------:|:-------------:|
|  DedocFileLoader   |           ❌           |       ❌       |
|   DedocPDFLoader   |           ❌           |       ❌       |
| DedocAPIFileLoader |           ❌           |       ❌       |

## 设置

* 要访问 `DedocFileLoader` 和 `DedocPDFLoader` 文档加载器，您需要安装 `dedoc` 集成包。
* 要访问 `DedocAPIFileLoader`，您需要运行 `Dedoc` 服务，例如 `Docker` 容器（更多详情请参阅[文档](https://dedoc.readthedocs.io/en/latest/getting_started/installation.html#install-and-run-dedoc-using-docker)）。

```bash
docker pull dedocproject/dedoc
docker run -p 1231:1231
```

`Dedoc` 的安装说明请参见[此处](https://dedoc.readthedocs.io/en/latest/getting_started/installation.html)。

In [1]:
# Install package
%pip install --quiet "dedoc[torch]"

Note: you may need to restart the kernel to use updated packages.


## 实例化

In [2]:
from langchain_community.document_loaders import DedocFileLoader

loader = DedocFileLoader("./example_data/state_of_the_union.txt")

## Load

In [3]:
docs = loader.load()
docs[0].page_content[:100]

'\nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and t'

## 惰性加载

In [4]:
docs = loader.lazy_load()

for doc in docs:
    print(doc.page_content[:100])
    break


Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and t


## API 参考

关于配置和调用 `Dedoc` 加载器的详细信息，请参阅 API 参考：

* https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.dedoc.DedocFileLoader.html
* https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.DedocPDFLoader.html
* https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.dedoc.DedocAPIFileLoader.html

## 加载任意文件

对于 [支持格式](https://dedoc.readthedocs.io/en/latest/#id1) 中任意文件的自动处理，
`DedocFileLoader` 会很有用。
文件加载器会自动检测具有正确扩展名的文件类型。

文件解析过程可以通过在 `DedocFileLoader` 类初始化期间设置的 `dedoc_kwargs` 进行配置。
这里给出了一些选项用法的基本示例，
请参阅 `DedocFileLoader` 文档和
[dedoc 文档](https://dedoc.readthedocs.io/en/latest/parameters/parameters.html)
以获取有关配置参数的更多详细信息。

### 基本示例

In [5]:
from langchain_community.document_loaders import DedocFileLoader

loader = DedocFileLoader("./example_data/state_of_the_union.txt")

docs = loader.load()

docs[0].page_content[:400]

'\nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\n\n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\n\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\n\n\nWith a duty to one another to the American people to '

### 分割模式

`DedocFileLoader` 支持将文档分割成不同类型 part（每个 part 单独返回）的功能。
为此，可以使用 `split` 参数，其选项如下：
* `document`（默认值）：文档文本将作为单个 langchain `Document` 对象返回（不分割）；
* `page`：将文档文本分割成页面（适用于 `PDF`、`DJVU`、`PPTX`、`PPT`、`ODP`）；
* `node`：将文档文本分割成 `Dedoc` 树节点（标题节点、列表项节点、纯文本节点）；
* `line`：将文档文本分割成文本行。

In [6]:
loader = DedocFileLoader(
    "./example_data/layout-parser-paper.pdf",
    split="page",
    pages=":2",
)

docs = loader.load()

len(docs)

2

### 处理表格

当在加载器初始化时将 `with_tables` 参数设置为 `True` 时（默认为 `with_tables=True`），`DedocFileLoader` 支持表格处理。

表格不会被拆分——每个表格对应一个 langchain `Document` 对象。
对于表格，`Document` 对象具有额外的 `metadata` 字段 `type="table"` 和 `text_as_html`，其中包含表格的 `HTML` 表示。

In [7]:
loader = DedocFileLoader("./example_data/mlb_teams_2012.csv")

docs = loader.load()

docs[1].metadata["type"], docs[1].metadata["text_as_html"][:200]

('table',
 '<table border="1" style="border-collapse: collapse; width: 100%;">\n<tbody>\n<tr>\n<td colspan="1" rowspan="1">Team</td>\n<td colspan="1" rowspan="1"> &quot;Payroll (millions)&quot;</td>\n<td colspan="1" r')

### 处理附件

`DedocFileLoader` 在初始化加载器时将 `with_attachments` 设置为 `True`（默认值为 `with_attachments=False`）时，支持处理附件。

附件根据 `split` 参数进行拆分。
对于附件，langchain 的 `Document` 对象有一个额外的元数据字段 `type="attachment"`。

In [8]:
loader = DedocFileLoader(
    "./example_data/fake-email-attachment.eml",
    with_attachments=True,
)

docs = loader.load()

docs[1].metadata["type"], docs[1].page_content

('attachment',
 '\nContent-Type\nmultipart/mixed; boundary="0000000000005d654405f082adb7"\nDate\nFri, 23 Dec 2022 12:08:48 -0600\nFrom\nMallori Harrell <mallori@unstructured.io>\nMIME-Version\n1.0\nMessage-ID\n<CAPgNNXSzLVJ-d1OCX_TjFgJU7ugtQrjFybPtAMmmYZzphxNFYg@mail.gmail.com>\nSubject\nFake email with attachment\nTo\nMallori Harrell <mallori@unstructured.io>')

## 加载 PDF 文件

如果您只想处理 `PDF` 文档，可以使用 `DedocPDFLoader` 并仅启用 `PDF` 支持。
该加载器支持文档拆分、表格和附件提取的相同参数。

`Dedoc` 可以提取带有或不带文本层的 `PDF`，
也可以自动检测其是否存在和正确性。
有几种 `PDF` 处理程序可供选择，您可以使用 `pdf_with_text_layer`
参数来选择其中一个。
请参阅 [参数说明](https://dedoc.readthedocs.io/en/latest/parameters/pdf_handling.html)
以获取更多详细信息。

对于不带文本层的 `PDF`，应安装 `Tesseract OCR` 及其语言包。
在这种情况下，[该说明](https://dedoc.readthedocs.io/en/latest/tutorials/add_new_language.html)会很有帮助。

In [9]:
from langchain_community.document_loaders import DedocPDFLoader

loader = DedocPDFLoader(
    "./example_data/layout-parser-paper.pdf", pdf_with_text_layer="true", pages="2:2"
)

docs = loader.load()

docs[0].page_content[:400]

'\n2\n\nZ. Shen et al.\n\n37], layout detection [38, 22], table detection [26], and scene text detection [4].\n\nA generalized learning-based framework dramatically reduces the need for the\n\nmanual speciﬁcation of complicated rules, which is the status quo with traditional\n\nmethods. DL has the potential to transform DIA pipelines and beneﬁt a broad\n\nspectrum of large-scale document digitization projects.\n'

## Dedoc API

如果您希望减少设置的复杂性就能立即开始使用，可以将 `Dedoc` 作为一项服务来使用。
**`DedocAPIFileLoader` 无需安装 `dedoc` 库即可使用。**
该加载器支持与 `DedocFileLoader` 相同的参数，并且
还能自动检测输入文件类型。

要使用 `DedocAPIFileLoader`，您需要运行 `Dedoc` 服务，例如使用 `Docker` 容器（请参阅[文档](https://dedoc.readthedocs.io/en/latest/getting_started/installation.html#install-and-run-dedoc-using-docker) 
了解更多详情）：

```bash
docker pull dedocproject/dedoc
docker run -p 1231:1231
```

请不要在您的代码中使用我们的演示 URL `https://dedoc-readme.hf.space`。

In [10]:
from langchain_community.document_loaders import DedocAPIFileLoader

loader = DedocAPIFileLoader(
    "./example_data/state_of_the_union.txt",
    url="https://dedoc-readme.hf.space",
)

docs = loader.load()

docs[0].page_content[:400]

'\nMadam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\n\n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\n\n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\n\n\nWith a duty to one another to the American people to '